pip install --user opencv-python

In [5]:
import sys
print(sys.version)

3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]


In [8]:
pip install --user mediapipe

  Using cached mediapipe-0.10.14-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached jax-0.4.30-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.30-cp311-cp311-win_amd64.whl.metadata (1.1 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.4.7-py3-none-win_amd64.whl.metadata (1.4 kB)
Using cached mediapipe-0.10.14-cp311-cp311-win_amd64.whl (50.8 MB)
Using cached sounddevice-0.4.7-py3-none-win_amd64.whl (200 kB)
Using cached jax-0.4.30-py3-none-any.whl (2.0 MB)
Using cached jaxlib-0.4.30-cp311-cp311-win_amd64.whl (51.9 MB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 웹캠 초기화
cap = cv2.VideoCapture(0)

# 참조 비디오 초기화 (실제로는 옆에 틀어져 있는 허리운동 영상을 사용해야 함)
reference_video = cv2.VideoCapture('reference_video.mp4')

def calculate_angle(a, b, c):
    """세 점 사이의 각도를 계산합니다."""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle

def get_pose_landmarks(image, pose):
    """이미지에서 포즈 랜드마크를 추출합니다."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    return results.pose_landmarks

def compare_poses(user_landmarks, reference_landmarks):
    """사용자의 포즈와 참조 포즈를 비교합니다."""
    if user_landmarks and reference_landmarks:
        # 예: 허리 각도 비교 (실제로는 더 많은 관절을 비교해야 함)
        user_angle = calculate_angle(
            [user_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x, user_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y],
            [user_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x, user_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y],
            [user_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x, user_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y]
        )
        reference_angle = calculate_angle(
            [reference_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x, reference_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y],
            [reference_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x, reference_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y],
            [reference_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x, reference_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y]
        )
        
        if abs(user_angle - reference_angle) > 15:  # 15도 이상 차이나면 피드백
            return f"허리 각도를 조정하세요. 현재: {user_angle:.2f}, 목표: {reference_angle:.2f}"
    return ""

while cap.isOpened() and reference_video.isOpened():
    # 웹캠에서 프레임 읽기
    success, image = cap.read()
    if not success:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break

    # 참조 비디오에서 프레임 읽기
    ref_success, ref_image = reference_video.read()
    if not ref_success:
        print("참조 비디오에서 프레임을 읽을 수 없습니다.")
        break

    # 포즈 추정
    user_landmarks = get_pose_landmarks(image, pose)
    reference_landmarks = get_pose_landmarks(ref_image, pose)

    # 포즈 비교 및 피드백
    feedback = compare_poses(user_landmarks, reference_landmarks)

    # 결과 표시
    mp_drawing.draw_landmarks(image, user_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.putText(image, feedback, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('MediaPipe Pose', image)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
reference_video.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [3]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 웹캠 초기화
cap = cv2.VideoCapture(0)

# 웹캠이 제대로 열렸는지 확인
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

while True:
    # 웹캠에서 프레임 읽기
    success, image = cap.read()
    if not success:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break

    # 이미지 처리
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    # 포즈 랜드마크 그리기
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # 결과 표시
    cv2.imshow('MediaPipe Pose', image)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

웹캠에서 프레임을 읽을 수 없습니다.


In [6]:
import cv2

# 사용 가능한 모든 카메라 장치를 확인
for i in range(10):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"카메라 {i}가 사용 가능합니다.")
        cap.release()
    else:
        print(f"카메라 {i}를 열 수 없습니다.")

# 특정 카메라 열기 (예: 카메라 0)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
else:
    ret, frame = cap.read()
    if ret:
        print("프레임을 성공적으로 읽었습니다.")
        cv2.imshow('Frame', frame)
        cv2.waitKey(0)
    else:
        print("프레임을 읽을 수 없습니다.")

cap.release()
cv2.destroyAllWindows()

카메라 0가 사용 가능합니다.
카메라 1를 열 수 없습니다.
카메라 2를 열 수 없습니다.
카메라 3를 열 수 없습니다.
카메라 4를 열 수 없습니다.
카메라 5를 열 수 없습니다.
카메라 6를 열 수 없습니다.
카메라 7를 열 수 없습니다.
카메라 8를 열 수 없습니다.
카메라 9를 열 수 없습니다.
프레임을 성공적으로 읽었습니다.


In [7]:
import cv2

cap = cv2.VideoCapture(0)

# 해상도 설정 (예: 640x480)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
else:
    ret, frame = cap.read()
    if ret:
        print("프레임을 성공적으로 읽었습니다.")
        print(f"프레임 크기: {frame.shape}")
        cv2.imshow('Frame', frame)
        cv2.waitKey(0)
    else:
        print("프레임을 읽을 수 없습니다.")

cap.release()
cv2.destroyAllWindows()

프레임을 성공적으로 읽었습니다.
프레임 크기: (480, 640, 3)


In [8]:
pip install opencv-python mediapipe numpy

Note: you may need to restart the kernel to use updated packages.


In [9]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 포즈 모듈 초기화
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 웹캠 초기화
cap = cv2.VideoCapture(0)

# MediaPipe Pose 모델 설정
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("웹캠에서 프레임을 읽을 수 없습니다.")
            continue

        # 성능 향상을 위해 이미지를 읽기 전용으로 설정
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # 이미지에 포즈 주석 그리기
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )

        # 프레임 표시
        cv2.imshow('MediaPipe Pose', image)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\master\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 랜드마크 이름 리스트
POSE_LANDMARKS = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
    'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
    'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("웹캠에서 프레임을 읽을 수 없습니다.")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )
            
            # 각 랜드마크의 위치와 이름을 화면에 표시
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                h, w, c = image.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(image, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
                cv2.putText(image, f'{idx}: {POSE_LANDMARKS[idx]}', (cx + 10, cy),
                            cv2.FONT_HERSHEY_PLAIN, 0.8, (0, 0, 255), 1)

        cv2.imshow('MediaPipe Pose with Landmarks', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()